### GetResult.py file finally

Function1:Determine CF or CB

Function2:get CB result(Embedding+BERD)

Function3:load CF model & return result

In [1]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl


In [2]:
from google.cloud import bigquery
from google.cloud import storage
import pickle
import pandas as pd
import surprise

In [3]:
def query_data(sql_query,user_id):
    client = bigquery.Client(project='brave-watch-414204')
    
    # Define query parameters
    query_params = [
        bigquery.ScalarQueryParameter("user_id", "INT64", int(user_id))
    ]

    # Execute the query job
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    query_job = client.query(sql_query, job_config=job_config)

    # Fetch the result
    result = query_job.result()

    return result

In [4]:
# Function 1
# Return 'CF' or 'CB' based on how many recipes did this user rate before
# input parm: user_id -> which will be send by the end application
def determine_rcs_model(user_id):
    # Initialize BigQuery client
    client = bigquery.Client(project='brave-watch-414204')

    sql_query = """
    SELECT COUNT(DISTINCT recipe_id) AS num_occurrences
    FROM `brave-watch-414204.RecipeQuery.interactions`
    WHERE user_id = @user_id
    """
    
    # Define query parameters
    query_params = [
        bigquery.ScalarQueryParameter("user_id", "INT64", int(user_id))
    ]

    # Execute the query job
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    query_job = client.query(sql_query, job_config=job_config)

    # Fetch the result
    result = query_job.result()
    num_occurrences = list(result)[0].num_occurrences

    if num_occurrences > 115:
        return 'CF'
    else:
        return 'CB'

In [5]:
# Function 3
# load the trained CF model from GCS Bucket -> the model is saved by the pipeline part
def get_CF_model(bucket_name,blob_name):
    
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    
    blob = bucket.blob(blob_name)
    
    pickle_file = blob.download_as_bytes()

    model = pickle.loads(pickle_file)
    
    return model

In [6]:
def process_CF_result(model,cust_user_id):
    # get recipes list
    sql_query_rated_recipe = """
        SELECT distinct recipe_id
        FROM `brave-watch-414204.RecipeQuery.interactions`
        WHERE user_id = @user_id
    """

    sql_query_all_recipe = """
        SELECT distinct recipe_id
        FROM `brave-watch-414204.RecipeQuery.interactions`
    """
    rated_recipes_id_result = query_data(sql_query_rated_recipe,cust_user_id)
    all_recipes_id_result = query_data(sql_query_all_recipe,cust_user_id)

    rated_recipes_id = []
    for row in rated_recipes_id_result:
        rated_recipes_id.append(row[0])

    all_recipes_id = []
    for row in all_recipes_id_result:
        all_recipes_id.append(row[0])

    unrated_recipes_list = list(set(all_recipes_id) - set(rated_recipes_id))

    predictions = []
    for recipe_id in unrated_recipes_list:
        prediction = model.predict(cust_user_id, recipe_id)
        predictions.append({'recipe_id': recipe_id, 'predicted_rating': prediction.est})

    predictions_df = pd.DataFrame(predictions)
    # Sort recipes by predicted ratings in descending order
    predictions_df = predictions_df.sort_values(by='predicted_rating', ascending=False)

    # get the recipes name and description
    sql_query_recipe_details = """
    SELECT  
      distinct id,
      name,
      description
    FROM `brave-watch-414204.RecipeQuery.recipes` 
    """

    recipes_detail_result = query_data(sql_query_recipe_details,'0')
    recipes_detail = recipes_detail_result.to_dataframe()

    final_predictions_df = pd.merge(predictions_df,recipes_detail,left_on='recipe_id',right_on='id',how='left')

    top5_rec = []
    for value in final_predictions_df.head().values:
        top5_rec.append({'id':value[0],'description':value[4],'name':value[3]})

    final_result = {"recipes":top5_rec}
    return final_result

In [7]:
#This part will be done by jh&th
def process_CB_results(cust_user_id):

    sql_result_processed_by_CB = """
        with tmp as (
            SELECT user_id,STRING_AGG(review, ' ') AS aggregated_reviews
            FROM `brave-watch-414204.RecipeQuery.interactions` 
            WHERE user_id = @user_id)
        ,
        
        review_embedding as (
            SELECT ml_generate_embedding_result,content AS query
            FROM
            ML.GENERATE_EMBEDDING(
                MODEL `RecipeQuery.gecko_model`,
                (SELECT aggregated_reviews as content, user_id as title FROM tmp),
                STRUCT(TRUE AS flatten_json_output)
                )
            )
        ,
        user_rated_recipe_id as(
            SELECT distinct recipe_id
            FROM `recipe-recommendation-2024.RecipeQuery.interactions`
            WHERE user_id = @user_id
        )
        ,
        recipe_similar_to_review_top_10 as (
        SELECT query.query,base.content,distance, base.title
        FROM VECTOR_SEARCH(
            TABLE `VectorDB.AfterEmbedding_768`, 'ml_generate_embedding_result',
            (
                SELECT * from review_embedding
                ),
                top_k => 10, options => '{"fraction_lists_to_search": 0.01}')
        )
        ,
        recipe_similar_to_review_top_5_exclude_rated_recipe as (
        select 
        base.title as recipe_id
        where base.title not in (select * from user_rated_recipe_id)
        limit 5
        )

        SELECT  
        id,
        name,
        description
        FROM `recipe-recommendation-2024.RecipeQuery.recipes` 
        where id in (select * from recipe_similar_to_review_top_5_exclude_rated_recipe)
"""

    query_job= query_data(sql_result_processed_by_CB,cust_user_id)

    results = query_job.result()

    final_result = results.to_dataframe()

    
    return final_result



In [8]:
# Integrated Function
# return the Final result
def main(cust_user_id):
    flag = determine_rcs_model(cust_user_id)
    if flag == 'CB':
        final_result = process_CB_results(cust_user_id)
        return final_result
    elif flag == 'CF':
        # get model
        model = get_CF_model('brave-watch-414204','Model/knn_model.pkl')
        final_result = process_CF_result(model,cust_user_id)
        
        return final_result
    else:
        raise ValueError(f"Unknown flag value: {flag}")
    return None

In [9]:
CF = main('8937')
CF

{'recipes': [{'id': 486261,
   'description': 'ready, set, cook! reynolds wrap contest entry. this recipe combines traditional mexican flavors and creates a fun "stack up" meal.  it uses reynolds wrap aluminum foil to steam off the skin of the poblano pepper, as well as tent the dish while baking.  this can be served as a main dish casserole or a yummy appetizer if sliced into small pieces.....great football food.  six servings as a main dish casserole or 12-15 appetizer sizes. if you like things hotter, add some diced jalapenos and garnish with jalapenos as well. enjoy!',
   'name': 'mexican stack up  rsc'},
  {'id': 487669,
   'description': '"ready, set, cook! reynolds wrap contest entry"  these easy to prepare, rosemary-sage rubbed ribs are grilled to "fall off the bone" tasty with the help of reynolds wrap foil.  paired with a spicy peach barbecue sauce, these ribs are smokin\'!',
   'name': 'just peachy grillin  ribs  rsc'},
  {'id': 495202,
   'description': 'ready, set, cook! h